### 1. Conexão ao banco de dados ###

In [3]:
import sqlite3
import pandas as pd
import os
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/falando_nela_v2/src')
    print("Current working directory (Colab):", os.getcwd())
else:
  print("Current working directory (not Colab):", os.getcwd())
DB_PATH = "../data/DiscursosSenadores.sqlite"
conn = sqlite3.connect(DB_PATH)

Mounted at /content/drive
Current working directory (Colab): /content/drive/MyDrive/falando_nela_v2/src


### 2. Pré-processamento dos textos ###


- remoção da palavra "Constituição" usada no jargão de processo legislativo e uniformização ortográfica
- alterações salvas no SQL

In [4]:
from filtros import filtros
substituicoes = {
         "Proposta de Emenda à Constituição": "PEC",
         "Comissão de Constituição e Justiça": "CCJ",
         "Comissão de Constituição, Justiça e Cidadania": "CCJ da Câmara",
         "constituicao": "Constituição",
         "constituiçao": "Constituição",
         "constituicão": "Constituição",

     }

filtros.substituir_expressoes(conn, "Discursos", "TextoIntegral", substituicoes)

### 3. Seleção dos discursos – 2007 em diante ###

In [5]:
conn.execute("DELETE FROM Discursos WHERE DataPronunciamento <= '2007-01-31'")
conn.commit()

### 4. Análise Exploratória de Dados ###

In [6]:
df = pd.read_sql_query("SELECT * FROM Discursos", conn)
from IPython.display import display
display(df.head())

,CodigoPronunciamento,CodigoParlamentar,DataPronunciamento,TextoResumo,TipoUsoPalavra,UrlTexto,Indexacao,SiglaPartidoParlamentarNaData,UfParlamentarNaData,SiglaCasaPronunciamento,Forma,CaminhoArquivo,DownloadConcluido,TextoIntegral
0,366328,825,2007-02-07,Regozijo pela indicação de S.Exa. à presidênci...,None,https://www25.senado.leg.br/web/atividade/pron...,"AGRADECIMENTO, INDICAÇÃO, ORADOR, PRESIDENTE, ...",PT,RS,SF,texto,../data/discursos/txt/366328.txt,1,O SR. PAULO PAIM (Bloco/PT - RS. Para uma comu...
1,366329,3,2007-02-07,Desafios do Programa de Aceleração do Crescime...,None,https://www25.senado.leg.br/web/atividade/pron...,"ELOGIO, PROGRAMA, ACELERAÇÃO, CRESCIMENTO, EXP...",PSB,SE,SF,texto,../data/discursos/txt/366329.txt,1,O SR. ANTÔNIO CARLOS VALADARES (Bloco/PSB - SE...
2,366330,3364,2007-02-07,Cobranças de ação federal contra a violência n...,None,https://www25.senado.leg.br/web/atividade/pron...,"CRITICA, INICIATIVA, PRESIDENTE DA REPUBLICA, ...",PMDB,PI,SF,texto,../data/discursos/txt/366330.txt,1,O SR. MÃO SANTA (PMDB - PI. Para uma comunicaç...
3,366331,4529,2007-02-07,Críticas ao recém-lançado Programa de Aceleraç...,None,https://www25.senado.leg.br/web/atividade/pron...,"COMENTARIO, ANUNCIO, PROGRAMA, ACELERAÇÃO, CRE...",PSDB,PB,SF,texto,../data/discursos/txt/366331.txt,1,O SR. CÍCERO LUCENA (PSDB - PB. Pronuncia o se...
4,366332,74,2007-02-07,Relato sobre a experiência de S.Exa. ao partic...,None,https://www25.senado.leg.br/web/atividade/pron...,"REGISTRO, PARTICIPAÇÃO, ORADOR, COMISSÃO, ACOM...",PFL,SP,SF,texto,../data/discursos/txt/366332.txt,1,O SR. ROMEU TUMA (PFL - SP. Pronuncia o seguin...


In [7]:
total_discursos = len(df)

df_filtrado = filtros.filtrar(conn, "Discursos", "TextoIntegral", ["Constituição"])
discursos_com_constituicao = len(df_filtrado)

proporcao = discursos_com_constituicao / total_discursos if total_discursos > 0 else 0

senadores_totais = df["CodigoParlamentar"].nunique()
senadores_filtrados = df_filtrado["CodigoParlamentar"].nunique()

print(f"Total de discursos: {total_discursos}")
print(f"Número de discursos com 'Constituição': {discursos_com_constituicao}")
print(f"Proporção: {proporcao:.2%}")
print(f"Número total de senadores: {senadores_totais}")
print(f"Número de senadores no que falaram na Constituição: {senadores_filtrados}")

Total de discursos: 67349
Número de discursos com 'Constituição': 12755
Proporção: 18.94%
Número total de senadores: 330
Número de senadores no que falaram na Constituição: 310
